#Embeddings

## Get embeddings for texts

## Get embeddings for images

In [22]:
import torch 
img = torch.randn(3,32,32)
print(img.shape)

torch.Size([3, 32, 32])


## Image to vector

In [23]:
# By conv

emb_size = 3
conv = torch.nn.Conv2d(3,emb_size,kernel_size= 16, stride = 16 )
out = conv(img.unsqueeze(0))
feature_map = out.squeeze(0)
print(feature_map.shape)
sequence = feature_map.flatten(1)
print(sequence.shape)


torch.Size([3, 2, 2])
torch.Size([3, 4])


In [ ]:
# By MLP
# patchify ...

fc = torch.nn.Linear(4,64,kernel_size= 8, stride = 8 )


How Linear layer process 2D input ?

In [24]:
input = torch.tensor([[1,2],[300,400]],dtype=torch.float)
fc = torch.nn.Linear(2,3)

out = fc(input.unsqueeze(0))
print(out.data,out.shape)


tensor([[[  -0.2147,   -0.4526,   -0.4532],
         [-114.1043, -122.3555,  -51.2447]]]) torch.Size([1, 2, 3])


-a batch with two dim

Let's create a sequence

In [25]:
sequence = sequence.permute(1,0)
print(sequence.shape) # sequence of 16 embedding size of each emb. is 128

torch.Size([4, 3])


Now process image embeddings by Linear layer

In [31]:
fc = torch.nn.Linear(emb_size,emb_size)
values = fc(sequence.unsqueeze(0)).squeeze(0)
print(values.shape) # Batch, sequence_len, emb_size

torch.Size([4, 3])


In MLP we apply non-linearity and repeat this step. But in attention layer we multiply outputs by attention weights and summ it

In [50]:
attention_weights  = torch.tensor([[ .0, .1, .1, .8],
      [ .5, .2, .2, .1],
      [ .3, .3, .2, .2]])



raw_out = []
for a in attention_weights:
  print(values.shape,a.shape)
  raw_out.append(values * a.T[:,None])
raw_out = torch.stack(raw_out)

print(raw_out, raw_out.shape)

#out = 

out = values[:,None] * attention_weights.T[:,:,None]
print(out)
print(out.shape)

assert torch.allclose(raw_out.reshape(4,3,3), out)


torch.Size([4, 3]) torch.Size([4])
torch.Size([4, 3]) torch.Size([4])
torch.Size([4, 3]) torch.Size([4])
tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 0.0801,  0.0638, -0.0355],
         [-0.0528, -0.0531, -0.0446],
         [ 0.6027,  0.7206, -0.7645]],

        [[ 0.2954,  0.0679,  0.1608],
         [ 0.1602,  0.1276, -0.0711],
         [-0.1055, -0.1062, -0.0892],
         [ 0.0753,  0.0901, -0.0956]],

        [[ 0.1772,  0.0408,  0.0965],
         [ 0.2403,  0.1913, -0.1066],
         [-0.1055, -0.1062, -0.0892],
         [ 0.1507,  0.1801, -0.1911]]], grad_fn=<StackBackward0>) torch.Size([3, 4, 3])
tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 0.2954,  0.0679,  0.1608],
         [ 0.1772,  0.0408,  0.0965]],

        [[ 0.0801,  0.0638, -0.0355],
         [ 0.1602,  0.1276, -0.0711],
         [ 0.2403,  0.1913, -0.1066]],

        [[-0.0528, -0.0531, -0.0446],
         [-0.1055, -0.1062, -0.0892],
         [-0.1055, -0.1062, -0.0892]],

        [[ 0.6027,  0.7206, -0.7645],

AssertionError: ignored

Другая проблемма сверток состоит в том что они слабо учитывают контекст.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/conv_vs_self_attention1.png"  width="700"></center>

Для нас очевидно что на кусте клубники скорее всего окажется еще одна ягода клубники а не малина, вишня или грузовик. 

In [ ]:
from torch.nn import MultiheadAttention

multihead_attn = nn.MultiheadAttention(embed_dim, num_heads = 1)
attn_output, attn_output_weights = multihead_attn(query = dummy_x, key = dummy_x , value =dummy_x)

for name, param in multihead_attn.named_parameters():
  print(name, param.shape)

print("Output shape",attn_output.shape)
print("Attention weights",attn_output_weights)

in_proj_weight torch.Size([768, 256])
in_proj_bias torch.Size([768])
out_proj.weight torch.Size([256, 256])
out_proj.bias torch.Size([256])
Output shape torch.Size([1, 16, 256])
Attention weights tensor([[[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]],

        [[1.]]], grad_fn=<MeanBackward1>)
